## Stratégie des Moyennes Mobiles Simples sur les Actions du S&P 500
Nous allons créer un Jupyter Notebook qui récupère les prix historiques des actions du S&P 500, calcule les moyennes mobiles pour chaque action, et applique la stratégie de croisement des moyennes mobiles simples.

In [6]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
import ssl #Importer le module ssl pour gérer la vérification du certificat SSL
import urllib.request
import sklearn
import math

##### Récupération de la table des actions du S&P 500

In [7]:
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE
urllib.request.install_opener(urllib.request.build_opener(urllib.request.HTTPSHandler(context=ssl_context)))

warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

##### Téléchargement des prix journaliers de l'ensemble des indices du S&P500 sur une période de 8 ans

In [4]:
today = dt.date.today()
yesterday = today - dt.timedelta(days=1)
end_date = yesterday.strftime('%Y-%m-%d')
start_date = pd.to_datetime(end_date) - pd.DateOffset(365 * 8)
df = yf.download(tickers=symbols_list, start=start_date, end=end_date, group_by='ticker')

[*********************100%%**********************]  503 of 503 completed


##### Transformation des données (stacking pour simplifier l'utilisation)

In [5]:
df = df.stack().reset_index()
df.columns = df.columns.str.lower()
print("Colonnes après reset_index : ", df.columns)

df = df.rename(columns={'level_0': 'ticker', 'level_1': 'date'})

df['ticker'] = df['ticker'].fillna('UNKNOWN')
df.fillna(0, inplace=True)

if 'ticker' in df.columns:
    # Remplacer '.' par '-' pour la colonne ticker
    df['ticker'] = df['ticker'].fillna('').astype(str).str.replace('.', '-', regex=False)  # Utilisation de fillna pour éviter les NaN

# Vérification des données
print(df.head())

Colonnes après reset_index :  Index(['date', 'price', 'gehc', 'carr', 'smci', 'vrsk', 'fang', 'lkq', 'apd',
       'ceg',
       ...
       'lulu', 'mu', 'now', 'hlt', 'eqt', 'gild', 'wba', 'emn', 'klac', 'met'],
      dtype='object', name='Ticker', length=505)


KeyError: 'ticker'

##### Fonction pour calculer les moyennes mobiles

In [49]:
# Calcul des moyennes mobiles courtes et longues sur les prix de clôture.

def calculate_sma(df, short_window=50, long_window=200):
    df['SMA50'] = df.groupby('ticker')['close'].apply(lambda x: x.rolling(window=short_window).mean())
    df['SMA200'] = df.groupby('ticker')['close'].apply(lambda x: x.rolling(window=long_window).mean())
    return df

##### Application de la stratégie des croisements de moyennes mobiles

In [50]:
# Signal d'achat lorsque SMA50 > SMA200, vente sinon.

def apply_trading_strategy(df):
    df['Signal'] = 0
    df['Signal'][df['SMA50'] > df['SMA200']] = 1  # Signal d'achat
    df['Signal'][df['SMA50'] < df['SMA200']] = -1  # Signal de vente
    return df

##### Visualisation des signaux de trading pour une entreprise spécifique

In [51]:
# Affiche les signaux de trading pour une action spécifique (ticker).

def plot_trading_signals(df, ticker):

    df_ticker = df[df['ticker'] == ticker].copy()

    plt.figure(figsize=(14, 7))
    plt.plot(df_ticker['date'], df_ticker['close'], label=f'Prix de clôture - {ticker}', color='blue')
    plt.plot(df_ticker['date'], df_ticker['SMA50'], label='SMA50', color='green')
    plt.plot(df_ticker['date'], df_ticker['SMA200'], label='SMA200', color='red')

    plt.fill_between(df_ticker['date'], df_ticker['close'], 
                     where=(df_ticker['Signal'] == 1), color='green', alpha=0.1, label='Achat')
    plt.fill_between(df_ticker['date'], df_ticker['close'], 
                     where=(df_ticker['Signal'] == -1), color='red', alpha=0.1, label='Vente')

    plt.title(f'Signaux de Trading pour {ticker} (SMA50 vs SMA200)')
    plt.legend(loc='upper left')
    plt.show()

##### Exécution du pipeline de trading sur les action du S&P500

In [52]:
# Exécution du pipeline de trading sur les action du S&P500
df = calculate_sma(df)  # Calcul des moyennes mobiles
df = apply_trading_strategy(df)  # Application de la stratégie
print(df[['date', 'ticker', 'close', 'SMA50', 'SMA200', 'Signal']].head())

KeyError: 'Column not found: close'

In [26]:
plot_trading_signals(df, 'AAPL')

KeyError: 'Requested level (ticker) does not match index name (None)'